In [7]:
pip install seaborn

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.32.3-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached huggingface_hub-0.32.3-py3-none-any.whl (512 kB)
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ----- ---------------------------------- 3.4/25.8 MB 15.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/25.8 MB 18.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/25.8 MB 18.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/25.8 MB 18.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/25.8 MB 18.5 MB/s eta 0:00:02
   ----------- ---------------------------- 7.3/25.8 MB 5.7 MB/s eta 0:00:04
   -------------- ----------

In [2]:
from huggingface_hub import hf_hub_download, list_repo_files
import os
import tarfile

repo_id = "xingjunm/WildDeepfake"
fake_dir = "deepfake_in_the_wild/fake_test"
real_dir = "deepfake_in_the_wild/real_test"
local_dir = "downloaded_data"
fake_images_folder = "data/test/fake"
real_images_folder = "data/test/real"

os.makedirs(local_dir, exist_ok=True)
os.makedirs(fake_images_folder, exist_ok=True)
os.makedirs(real_images_folder, exist_ok=True)

files = list_repo_files(repo_id, repo_type="dataset")

for file_path in files:
    target_folder = None
    if file_path.startswith(fake_dir):
        target_folder = fake_dir
        output_folder = fake_images_folder
    elif file_path.startswith(real_dir):
        target_folder = real_dir
        output_folder = real_images_folder

    if target_folder:
        local_file_path = os.path.join(local_dir, file_path[len(target_folder) + 1:])
        try:
            print(f"Скачиваем {file_path}...")
            download_path = hf_hub_download(repo_id, filename=file_path, repo_type="dataset", local_dir=local_dir)
            file_size = os.path.getsize(download_path)
            print(f"  Файл скачан, размер: {file_size} байт")

            if file_path.endswith(".tar.gz"):
                print(f"  Распаковываем {file_path} как tar (без gzip)...")
                try:
                    with tarfile.open(download_path, 'r') as tar_ref:
                        for member in tar_ref.getmembers():
                            if not member.isdir() and any(member.name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.bmp']):
                                try:
                                    image_file = tar_ref.extractfile(member)
                                    if image_file:
                                        image_data = image_file.read()
                                        local_image_path = os.path.join(output_folder, os.path.basename(member.name))
                                        with open(local_image_path, "wb") as f:
                                            f.write(image_data)
                                        #print(f"    Извлечено изображение: {member.name}")
                                except Exception as e:
                                    print(f"    Ошибка при извлечении изображения {member.name}: {e}")
                except Exception as e:
                    print(f"  Ошибка при распаковке архива {file_path}: {e}")

                # Удаление архива после распаковки
                try:
                    os.remove(download_path)
                    print(f"  Архив {file_path} удален.")
                except OSError as e:
                    print(f"  Ошибка при удалении архива {file_path}: {e}")
            else:
                print(f"  Файл {file_path} не является архивом tar.gz, пропущен")

            print(f"  Файл {file_path} обработан.")

        except Exception as e:
            print(f"Ошибка при скачивании файла {file_path}: {e}")

print("Загрузка и распаковка завершены.")

Скачиваем deepfake_in_the_wild/fake_test/1.tar.gz...
  Файл скачан, размер: 69683200 байт
  Распаковываем deepfake_in_the_wild/fake_test/1.tar.gz как tar (без gzip)...
  Архив deepfake_in_the_wild/fake_test/1.tar.gz удален.
  Файл deepfake_in_the_wild/fake_test/1.tar.gz обработан.
Скачиваем deepfake_in_the_wild/fake_test/10.tar.gz...
  Файл скачан, размер: 14592000 байт
  Распаковываем deepfake_in_the_wild/fake_test/10.tar.gz как tar (без gzip)...
  Архив deepfake_in_the_wild/fake_test/10.tar.gz удален.
  Файл deepfake_in_the_wild/fake_test/10.tar.gz обработан.
Скачиваем deepfake_in_the_wild/fake_test/100.tar.gz...
  Файл скачан, размер: 28221440 байт
  Распаковываем deepfake_in_the_wild/fake_test/100.tar.gz как tar (без gzip)...
  Архив deepfake_in_the_wild/fake_test/100.tar.gz удален.
  Файл deepfake_in_the_wild/fake_test/100.tar.gz обработан.
Скачиваем deepfake_in_the_wild/fake_test/101.tar.gz...
  Файл скачан, размер: 4106240 байт
  Распаковываем deepfake_in_the_wild/fake_test/101.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator

from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D, BatchNormalization, LeakyReLU, Concatenate, MaxPooling2D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import glob
from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import EfficientNetB4
import cv2
from tensorflow.keras.preprocessing import image

In [13]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import os
import random
import shutil

def delete_random_images(folder_path, num_to_keep):

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    num_images = len(image_files)

    if num_images <= num_to_keep:
        print(f"В папке {folder_path} меньше или равно {num_to_keep} изображений. Удаление не требуется.")
        return

    num_to_delete = num_images - num_to_keep
    files_to_delete = random.sample(image_files, num_to_delete)

    for file in files_to_delete:
        file_path = os.path.join(folder_path, file)
        try:
            os.remove(file_path)
            print(f"Удалено: {file_path}")
        except OSError as e:
            print(f"Ошибка при удалении {file_path}: {e}")



Удалено: OF/test/real\real_2303.jpg
Удалено: OF/test/real\real_2081.jpg
Удалено: OF/test/real\real_3269.jpg
Удалено: OF/test/real\real_4575.jpg
Удалено: OF/test/real\real_53.jpg
Удалено: OF/test/real\real_872.jpg
Удалено: OF/test/real\real_1367.jpg
Удалено: OF/test/real\real_1304.jpg
Удалено: OF/test/real\real_4035.jpg
Удалено: OF/test/real\real_1154.jpg
Удалено: OF/test/real\real_3651.jpg
Удалено: OF/test/real\real_3483.jpg
Удалено: OF/test/real\real_897.jpg
Удалено: OF/test/real\real_1313.jpg
Удалено: OF/test/real\real_2326.jpg
Удалено: OF/test/real\real_4631.jpg
Удалено: OF/test/real\real_1701.jpg
Удалено: OF/test/real\real_2061.jpg
Удалено: OF/test/real\real_911.jpg
Удалено: OF/test/real\real_3501.jpg
Удалено: OF/test/real\real_4436.jpg
Удалено: OF/test/real\real_2644.jpg
Удалено: OF/test/real\real_1839.jpg
Удалено: OF/test/real\real_4853.jpg
Удалено: OF/test/real\real_2487.jpg
Удалено: OF/test/real\real_3665.jpg
Удалено: OF/test/real\real_3363.jpg
Удалено: OF/test/real\real_1655.j

In [ ]:
TOKEN = '' 
CHAT_ID = ''
# -------------------------------

bot = telebot.TeleBot(TOKEN)

def send_completion_message(message):
  try:
    bot.send_message(CHAT_ID, message)
    print("Сообщение об окончании работы отправлено в Telegram.")
  except Exception as e:
    print(f"Ошибка при отправке сообщения в Telegram: {e}")

if __name__ == "__main__":
  try:
    completion_message = f"Код завершен!"
    send_completion_message(completion_message)

  except Exception as e:
    error_message = f"Произошла ошибка во время выполнения кода: {e}"
    send_completion_message(error_message) 

  finally:
    print("Программа завершила работу.")